In [ ]:
!pip install numpy pytest
#!pip install onnx onnxruntime 
!pip install --pre -f https://onnxruntimepackages.z14.web.core.windows.net/onnx-function-experiment.html onnx-function-experiment
!pip install --pre -f https://onnxruntimepackages.z14.web.core.windows.net/onnxruntime-function-experiment.html ort-function-experiment-nightly

In [ ]:
!pip install -e .

In [2]:
"""
Generating a ModelProto
=======================

This example demonstrates the use of *onnx-script* to define an ONNX model.
*onnx-script* behaves like a compiler. It converts a script into an ONNX model.

"""

#%%
# First, we define the implementation of a square-loss function in onnxscript.

import numpy as np
import onnx
from onnxruntime import InferenceSession

from onnxscript import FLOAT
from onnxscript import opset15 as op
from onnxscript import proto2text, script


@script()
def square_loss(X: FLOAT["N", 1], Y: FLOAT["N", 1]) -> FLOAT[1, 1]:  # noqa: F821
    diff = X - Y
    return op.ReduceSum(diff * diff, keepdims=1)


#%%
# We can convert it to a model (an ONNX *ModelProto*) as follows:

model = square_loss.to_model_proto()

#%%
# Let's see what the generated model looks like.
print(proto2text(model))

#%%
# We can run shape-inference and type-check the model using the standard ONNX API.

# model = onnx.shape_inference.infer_shapes(model)
# onnx.checker.check_model(model)

#%%
# And finally, we can use *onnxruntime* to compute the outputs
# based on this model, using the standard onnxruntime API.

sess = InferenceSession(model.SerializeToString())

X = np.array([[0, 1, 2]], dtype=np.float32).T
Y = np.array([[0.1, 1.2, 2.3]], dtype=np.float32).T

got = sess.run(None, {"X": X, "Y": Y})
expected = ((X - Y) ** 2).sum()

print(expected, got)

ModuleNotFoundError: No module named 'onnx.defs'

Bad pipe message: %s [b"\xfa\xa2\xa2\xf4\x17)\xe8'\x02K*\x08\xfa\xee\xb4\xc3\xda( \xb4Q}\x17k\x07\xdb\xe9A-\xcd\x0bh\x1by\xf3\x00>j\x91\x1d\x1a\x1a`\x137Z1\xe6\xcc/\x9f\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n", b'\x08\x04\x08\x05\x08\x06\x04', b'', b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 A~\t\xd1q\xd6\x0cg\xc6\xfaz\x14Q^h\xd7~\x0b\xca\xe8z\x95']
Bad pipe message: %s [b'\x1b}7Yt|c;\xb2\x16\xee\xbc\xbcVo\xe5\r\xd3\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R']
Bad pipe message: %s [b'u\x07\xd68-P\x9a6

In [ ]:
!wget https://raw.githubusercontent.com/onnx/onnx/f1cd4167d5ffd6e0ffbb2e9d58e8e831aeebb428/onnx/onnx.in.proto
!sed 's/{PACKAGE_NAME}/onnx/g' onnx.in.proto
!wget https://raw.githubusercontent.com/onnx/onnx/f1cd4167d5ffd6e0ffbb2e9d58e8e831aeebb428/onnx/onnx-data.in.proto

In [ ]:
# NOW MANUALLY ADD THE CODE FROM onnx-data.in.proto TO onnx.in.proto (without the `package`/`syntax` stuff)

In [ ]:
!pip install grpcio-tools # just an easy way to install protoc, which we need to compile the proto files into python data class thingies

In [11]:
!pip install "betterproto[compiler]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 2.0 MB/s eta 0:00:00ta 0:00:01


In [2]:
!mkdir onnxproto
!python -m grpc_tools.protoc -I=. --python_betterproto_out=onnxproto onnx.in.proto

mkdir: cannot create directory ‘onnxproto’: File exists
Writing __init__.py
Writing onnx.py


In [ ]:
# NOW MANUALLY MOVE THE CONTENTS OF onnxproto/onnx.py TO onnx/onnxproto.py

In [2]:
!mkdir onnxprotogoogle
!python -m grpc_tools.protoc -I=. --python_out=onnxprotogoogle onnx.in.proto

In [2]:
!cd onnx && wget https://raw.githubusercontent.com/onnx/onnx/f1cd4167d5ffd6e0ffbb2e9d58e8e831aeebb428/onnx/numpy_helper.py

--2023-01-05 04:23:24--  https://raw.githubusercontent.com/onnx/onnx/f1cd4167d5ffd6e0ffbb2e9d58e8e831aeebb428/onnx/numpy_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14129 (14K) [text/plain]
Saving to: ‘numpy_helper.py’

numpy_helper.py     100%[===================>]  13.80K  --.-KB/s    in 0.001s  

2023-01-05 04:23:24 (26.1 MB/s) - ‘numpy_helper.py’ saved [14129/14129]

